In [2]:
import tensorflow as tf
import cv2
import torch
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

2023-05-20 06:13:42.477161: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-20 06:13:44.039421: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-20 06:13:44.041230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 06:13:53.483339: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
loaded_model = tf.keras.models.load_model('model1.h5')

2023-05-20 06:14:22.938936: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 293601280 exceeds 10% of free system memory.
2023-05-20 06:14:23.143159: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 293601280 exceeds 10% of free system memory.
2023-05-20 06:14:23.263759: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 293601280 exceeds 10% of free system memory.
2023-05-20 06:14:39.448086: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 293601280 exceeds 10% of free system memory.
2023-05-20 06:14:40.053915: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 293601280 exceeds 10% of free system memory.


In [10]:
# image = cv2.imread('plastic bottle.jpg')
# img = cv2.imread('glass.jpg')
# img_array = img_to_array(img)
# image = np.expand_dims(img_array, axis=0)
# normalized_img = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
# prediction = loaded_model.predict(normalized_img)

1/1 [==============================] - 0s 490ms/step


In [3]:
labels = ['cardboard', 'glass', 'metal', 'paper', 'cardboard', 'trash']
img = cv2.imread('dataset/train/glass/glass1.jpg')
resized_img = cv2.resize(img, (512, 384)) 
normalized_img = resized_img.astype(np.float32) / 255.0 
# input_data = np.expand_dims(normalized_img, axis=0)
# predictions = loaded_model.predict(input_data)
# class_index = np.argmax(predictions)
# predicted_label = labels[class_index]

In [1]:
cv2.imshow('Input Image', normalized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [13]:
print(predicted_label)

cardboard


In [12]:
labels = ['cardboard', 'glass', 'metal', 'paper', 'cardboard', 'trash']

# Convert probabilities to labels
# predicted_label_index = np.argmax(prediction)
# predicted_label = labels[predicted_label_inde]
predicted_label = labels[np.argmax(prediction)]

print(predicted_label)

cardboard


In [ ]:
cap = cv2.VideoCapture(3)
while True:
    ret, frame = cap.read()

    output = model(frame)

    cv2.imshow('Live Prediction', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
